In [1]:
# -*- encoding: utf-8 -*-
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
import os
from multiprocessing import pool, cpu_count
from pprint import pprint
from matplotlib import pyplot as plt
from pprint import pprint


# get 4326时四点

In [4]:
import osmnx as ox
%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

'0.10'

In [6]:
G = ox.graph_from_place('上海市,China',which_result=2,simplify=False,
     custom_filter=('["area"!~"yes"]["highway"~"ringway|primary"]'
                                      '["service"!~"parking|parking_aisle|driveway|private|emergency_access"]'
                                      '["motor_vehicle"!~"no"]["motorcar"!~"no"]'
                                      )
                        )

In [ ]:
fig, ax = ox.plot_graph(G, edge_linewidth=2, node_size=10)

In [ ]:
nodes_proj, edges_proj = ox.graph_to_gdfs(G)
nodes_proj.head()

In [7]:
# Get the x coordinates of the nodes
nodes_proj['x'] = nodes_proj["x"].astype(float)

In [8]:
from pyproj import CRS
CRS(nodes_proj.crs).to_epsg()

4326

In [ ]:
# Get the x coordinates of the nodes
nodes_proj['x'] = nodes_proj["x"].astype(float)

# 最西
# Retrieve the maximum x value (i.e. the most eastern)
maxx = nodes_proj['x'].min()
print(maxx)
target = nodes_proj.loc[nodes_proj['x']==maxx, 'geometry'].values[0]
print(target)

#最东
max1 = nodes_proj['x'].max()
print(max1)
target = nodes_proj.loc[nodes_proj['x']==max1, 'geometry'].values[0]
print(target)

#最南
max2 = nodes_proj['y'].min()
print(max2)
target = nodes_proj.loc[nodes_proj['y']==max2, 'geometry'].values[0]
print(target)

#最北
max3 = nodes_proj['y'].max()
print(max3)
target = nodes_proj.loc[nodes_proj['y']==max3, 'geometry'].values[0]
print(target)

In [ ]:
map_src = Path('E:\Program Files\jupyter_code\code/thinning.png')
map_table = cv2.imread(str(map_src), 0)

[rows, cols] = map_table.shape
print(rows, cols)
a1 = 0
a2 = 1999
b3 = 0
b4 = 1999
for i in range(rows):
    for j in range(cols):
        if map_table[i, j]>100:
            if(i>a1):
                a1 = i
                b1 = j
            if(i<a2):
                a2 = i
                b2 = j
            if(j>b3):
                a3 = i
                b3 = j
            if(j<b4):
                a4 = i
                b4 = j
print(a1,b1)#最南
print(a2,b2)#最北
print(a3,b3)#最东
print(a4,b4)#最西


In [3]:
# 1800 2192
# 1764 828最南
# 34 733最北
# 954 1800最东
# 1392 175最西

# 121.0047308 30.8856365最西
# 121.8684836 30.8981152最东

# 121.3490946 30.7177534最南
# 121.3004672 31.5001065最北

# 309274.1950475702 3418632.843091207
# 401795.000767322 3418943.145763395

# 334350.7659118437 3398258.160110353
# 338594.3769534618 3486279.460481172

gpsX = np.array([[121.3490946, 121.0047308,], [828,175]])
gpsY = np.array([[30.7177534,30.8856365], [1764,1392]])

diffX = gpsX[:, 1] - gpsX[:, 0]
kx = diffX[1] / diffX[0]
bx = gpsX[1, 0] - gpsX[0, 0] * kx

diffY = gpsY[:, 1] - gpsY[:, 0]
ky = diffY[1] / diffY[0]
by = gpsY[1, 0] - gpsY[0, 0] * ky
print(kx, bx) 
print(ky, by)

1896.250418888416 -229280.27146698005
-2215.827560963539 69829.24459460145


In [7]:
lon,lat = 121.3004672,31.5001065
coordX = (kx * lon + bx)
coordY = (ky * lat + by)
print(coordX,coordY)

735.7902723805164 30.440438614736195


In [26]:
def gps2xy(lon, lat):
    # GPS coordinate: 121.0047308 30.8856365最西 121.9724096 30.8997416最东 121.2702538 30.7056483最南 121.3004672 31.5001065最北
    # raster_loc = 1766 685最南 32 742最北 1340 1995最东 1365 192最西
    
    # calculate coefficient k and b
    """
    gpsX = np.array([[103.9925022, 104.088888], [688, 1729]])
    gpsY = np.array([[30.7097699, 30.590786], [856, 2144]])

    diffX = gpsX[:, 1] - gpsX[:, 0]
    kx = diffX[1] / diffX[0]
    bx = gpsX[1, 0] - gpsX[0, 0] * kx

    diffY = gpsY[:, 1] - gpsY[:, 0]
    ky = diffY[1] / diffY[0]
    by = gpsY[1, 0] - gpsY[0, 0] * ky
    print(kx, bx) 
    print(ky, by)
    """
    kx = 1896.250418888416 
    bx = -229280.27146698005
    ky = -2215.827560963539
    by = 69829.24459460145
 
 
    coordX = np.round(kx * lon + bx).astype(int)
    coordY = np.round(ky * lat + by).astype(int)
    # print(coordX, coordY)
    return (coordX, coordY) 
 